In [1]:
from __future__ import annotations

import operator
import re
from collections import Counter, defaultdict, deque
from dataclasses import dataclass
from itertools import combinations, repeat
from typing import List, Tuple
from functools import cache

import black
import jupyter_black
import networkx as nx
from numba import jit
from parse import parse

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def first(iterable):
    return next(iter(iterable))

In [121]:
# Day 1: Inverse Captcha
def match(digits, offset=1):
    for i, a in enumerate(digits):
        if a == digits[(i + offset) % len(digits)]:
            yield a


digits = open("2017/1.txt").read().strip()
digits = [int(x) for x in digits]

print("Part 1:", sum(match(digits)))  # 1203
print("Part 2:", sum(match(digits, offset=len(digits) // 2)))  # 1146

Part 1: 1203
Part 1: 1146


In [149]:
# Day 2: Corruption Checksum
def evenly_divides(numbers: List[int]) -> int:
    for a, b in combinations(sorted(numbers), 2):
        if b / a == b // a:
            return b // a


lines = open("2017/2.txt").read().strip().splitlines()
spreadsheet = [[int(x) for x in line.split()] for line in lines]
print("Part 1:", sum(max(row) - min(row) for row in spreadsheet))  # 32121
print("Part 2:", sum(evenly_divides(row) for row in spreadsheet))  # 197

Part 1: 32121
Part 2: 197


In [41]:
# Day 3: Spiral Memory
def spiral_move():
    #                         58  57
    # 37  36  35  34  33  32  31  56
    # 38  17  16  15  14  13  30  55
    # 39  18   5   4   3  12  29  54
    # 40  19   6   1   2  11  28  53
    # 41  20   7   8   9  10  27  52
    # 42  21  22  23  24  25  26  51
    # 43  44  45  46  47  48  49  50

    # How to generate:
    # r u l l d d r r r (10) u u u l l l l (17) d d d d (21) r r r r r (26) u u u u u (31)
    # l l l l l l (37) d d d d d d (43) r r r r r r r (50) u u u u u u u (57)

    # 1R, 1U, 2L, 2D, 3R, 3U, 4L, 4D, 5R, 5U, 6L, 6D, 7R, 7U,...

    def move(dirs):
        nonlocal x, y, steps
        for dir in dirs:
            for dy, dx in repeat(dir, steps):
                x += dx
                y += dy
                yield (x, y)
        steps += 1

    right_up = ((1, 0), (0, -1))
    left_down = ((-1, 0), (0, 1))
    steps = 1
    x = y = 0
    yield (x, y)
    while True:
        yield from move(right_up)
        yield from move(left_down)


def distance(N=277678):
    for number, node in enumerate(spiral_move(), 1):
        if number == N:
            return sum(abs(coord) for coord in node)


def value_exceeds(N=277678):
    # fmt:off
    def neighbors8(point): 
        "The eight neighboring squares."
        x, y = point 
        return ((x-1, y-1), (x, y-1), (x+1, y-1),
                (x-1, y),             (x+1, y),
                (x-1, y+1), (x, y+1), (x+1, y+1))
    # fmt:on

    memory = {}
    for node in spiral_move():
        number = sum(memory[n] for n in neighbors8(node) if n in memory) or 1
        memory[node] = number
        if number > N:
            return number


print("Part 1:", distance())  # 475
print("Part 2:", value_exceeds())  # 279138

Part 1: 475
Part 2: 279138


In [19]:
# Day 4: High-Entropy Passphrases
def valid1(line):
    return len(line.split()) == len(set(line.split()))


def valid2(line):
    words = [''.join(sorted(x)) for x in line.split()]
    return len(words) == len(set(words))


lines = open("2017/4.txt").read().strip().splitlines()
print("Part 1:", sum(valid1(line) for line in lines)) # 386
print("Part 2:", sum(valid2(line) for line in lines)) # 208

Part 1: 386
Part 2: 208


In [47]:
# Day 5: A Maze of Twisty Trampolines, All Alike
def run(instructions, part2=False):
    pc = steps = 0
    while pc < len(instructions):
        offset = instructions[pc]
        if part2 and offset >= 3:
            instructions[pc] -= 1
        else:
            instructions[pc] += 1
        steps += 1
        pc += offset
    return steps

instructions = open('2017/5.txt').read().strip().splitlines()
instructions = [int(x) for x in instructions]

print('Part 1:', run(instructions[:])) # 342669
print('Part 2:', run(instructions, part2=True)) # 25136209


Part 1: 342669
Part 2: 25136209


In [76]:
# Day 6: Memory Reallocation
def redistribute(banks: Tuple[int, ...]) -> Tuple[int, ...]:
    banks = list(banks)
    blocks = max(banks)
    index = banks.index(blocks)
    banks[index] = 0
    while blocks:
        index = (index + 1) % len(banks)
        banks[index] += 1
        blocks -= 1
    return tuple(banks)


def cycle_unique(banks: Tuple[int, ...]) -> Tuple[int, int]:
    seen = {}
    cycles = 0
    while banks not in seen:
        seen[banks] = cycles
        banks = redistribute(banks)
        cycles += 1
    return cycles, seen[banks]


banks = tuple(int(x) for x in open("2017/6.txt").read().strip().split())

cycles, first_seen = cycle_unique(banks)
print("Part 1:", cycles)  # 4074
print("Part 2:", cycles - first_seen)  # 2793

Part 1: 4074
Part 2: 2793


In [12]:
# Day 7: Recursive Circus
def parse_lines(lines):
    programs = {}
    children = {}
    parent = {}
    # Parse
    for line in lines:
        items = line.split(" -> ")
        name, weight = items[0].split()
        weight = int(re.search("\d+", weight)[0])
        programs[name] = weight
        if len(items) > 1:
            children[name] = set(items[1].split(", "))
            for child in children[name]:
                parent[child] = name
    return programs, children, parent


def branch_weight(node):
    if node not in children:
        return programs[node]
    return sum(branch_weight(child) for child in children[node]) + programs[node]


def unbalanced_weight(node):
    "Returns weight of unbalanced branch, 0 if all branches are balanced."
    branch_weights = Counter(branch_weight(child) for child in children[node])
    if len(branch_weights) == 1:
        return 0
    return branch_weights.most_common()[-1][0]


def offset(node):
    "Weight needed to adjust node to balance all branches."
    branch_weights = Counter(branch_weight(child) for child in children[parent[node]])
    return branch_weights.most_common()[0][0] - branch_weights.most_common()[1][0]


def node_with_weight(weight):
    "Returns node where branch weight equals `weight`"
    for node in programs:
        if branch_weight(node) == weight:
            return node


def unbalanced_node(node):
    while unbalanced_weight(node):
        node = node_with_weight(unbalanced_weight(node))
    return node


lines = open("2017/7.txt").read().strip().splitlines()
programs, children, parent = parse_lines(lines)
root = first(program for program in programs if program not in parent)
print("Part 1:", root)  # hlhomy
node = unbalanced_node(root)
print("Part 2:", programs[node] + offset(node))  # 1505

Part 1: hlhomy
Part 2: 1505


In [17]:
# Day 8: I Heard You Like Registers
def max_register(lines):
    operations = {
        ">": operator.gt,
        "<": operator.lt,
        ">=": operator.ge,
        "<=": operator.le,
        "==": operator.eq,
        "!=": operator.ne,
    }

    registers = defaultdict(int)
    max_seen = 0
    for line in lines:
        p = parse("{} {} {:d} if {} {} {:d}", line)
        target, mod, steps, register, comparison, value = p
        if operations[comparison](registers[register], value):
            if mod == "inc":
                registers[target] += steps
            elif mod == "dec":
                registers[target] -= steps
            else:
                raise ValueError("Invalid modifier", mod)
        max_seen = max(registers[target], max_seen)
    return max(registers.values()), max_seen


lines = open("2017/8.txt").read().strip().splitlines()
current_max, max_seen = max_register(lines)
print("Part 1:", current_max)  # 7296
print("Part 2:", max_seen)  # 8186

Part 1: 7296
Part 2: 8186


In [75]:
# Day 9: Stream Processing
def parse(stream):
    open_parens = score = garbage_count = 0
    garbage = False
    it = iter(stream)
    for char in it:
        match char:
            case "<":
                garbage = True
            case ">":
                garbage = False
                # # Opening garbage '<' will add to garbage_count, remove it when we close garbage
                garbage_count -= 1
            case "!":
                next(it)
                continue
        if garbage:
            garbage_count += 1
        else:
            match char:
                case "{":
                    open_parens += 1
                case "}":
                    score += open_parens
                    open_parens -= 1
    return score, garbage_count


stream = open("2017/9.txt").read().strip()
score, garbage = parse(stream)
print("Part 1:", score)  # 11347
print("Part 2:", garbage)  # 5404

Part 1: 11347
Part 2: 5404


In [3]:
# Day 10: Knot Hash
def one_round(lengths, circle):
    global start, skip
    for length in lengths:
        stop = start + length
        sub_circle = [circle[x % len(circle)] for x in range(start, stop)]
        sub_circle.reverse()
        for i, x in enumerate(range(start, stop)):
            circle[x % len(circle)] = sub_circle[i]
        start += (length + skip) % len(circle)
        skip += 1


def xor_hash(sparse_hash):
    "Returns a list of 16 numbers created by XOR:ing chunks of 16 numbers in sparse_hash. sparse_hash needs to be 256 long"
    assert len(sparse_hash) == 256

    def xor_block(block):
        "Xor a list of 16 ints"
        result = 0
        for x in block:
            result ^= x
        return result

    result = []
    for chunk in range(0, 256, 16):
        result.append(xor_block(sparse_hash[chunk : chunk + 16]))

    return result


def knot_hash(ascii_input):
    global skip, start
    skip = start = 0

    circle = list(range(256))
    lengths = [ord(x) for x in ascii_input] + [17, 31, 73, 47, 23]
    for _ in range(64):
        one_round(lengths, circle)
    numbers = xor_hash(circle)
    return bytes(numbers).hex()


circle = list(range(256))
skip = start = 0
lengths = [88, 88, 211, 106, 141, 1, 78, 254, 2, 111, 77, 255, 90, 0, 54, 205]
one_round(lengths, circle)
print("Part 1:", circle[0] * circle[1])  # 11375
# Part 2
print(
    "Part 2:", knot_hash("88,88,211,106,141,1,78,254,2,111,77,255,90,0,54,205")
)  # e0387e2ad112b7c2ef344e44885fe4d8

Part 1: 11375
Part 2: e0387e2ad112b7c2ef344e44885fe4d8


In [4]:
# Day 11: Hex Ed
def reduce_directions():
    def reduce_opposites(a, b):
        if steps[a] > steps[b]:
            steps[a] -= steps[b]
            del steps[b]
        else:
            steps[b] -= steps[a]
            del steps[a]

    def two_to_one(a1, a2, b):
        while steps[a1] > 0 and steps[a2] > 0:
            steps[a1] -= 1
            steps[a2] -= 1
            steps[b] += 1
    
    reduce_opposites("sw", "ne")
    reduce_opposites("se", "nw")
    reduce_opposites("s", "n")
    # sw + se -> s, etc.
    two_to_one("sw", "se", "s")
    two_to_one("nw", "ne", "n")
    two_to_one("ne", "s", "se")
    two_to_one("nw", "s", "sw")
    two_to_one("se", "n", "ne")
    two_to_one("sw", "n", "nw")


def distance(steps):
    return sum(steps.values())


line = open("2017/11.txt").read().strip().split(",")
longest = 0
steps = Counter()
for step in line:
    steps.update([step])
    reduce_directions()
    longest = max(distance(steps), longest)

print("Part 1:", distance(steps))  # 650
print("Part 2:", longest)  # 1465

Part 1: 650
Part 2: 1465


In [96]:
# Day 12: Digital Plumber
def group(pipes, target):
    "Returns a sorted tuple of all programs that can see `target`"
    can_see = {target}
    frontier = [target]
    while frontier:
        current = frontier.pop()

        for neighbor in pipes[current]:
            if neighbor not in can_see:
                frontier.append(neighbor)
                can_see.add(neighbor)
    return tuple(sorted(can_see))


lines = open("2017/12.txt").read().strip().splitlines()
pipes = {
    i: set(int(x) for x in re.findall(r"\d+", line.split(" <-> ")[1]))
    for i, line in enumerate(lines)
}

print("Part 1:", len(group(pipes, 0)))  # 306

all_groups = set()
for target in pipes:
    all_groups.add(group(pipes, target))
print("Part 2:", len(all_groups))  # 200

# Alternative implementation with networkx from
# https://www.reddit.com/r/adventofcode/comments/7j89tr/comment/dr4fxul/ 
graph = nx.Graph()
for line in lines:
    node, neighbors = line.split(" <-> ")
    graph.add_edges_from((node, neighbor) for neighbor in neighbors.split(", "))
# print("Part 1:", len(nx.node_connected_component(graph, "0")))
# print("Part 2:", nx.number_connected_components(graph))

Part 1: 306
Part 2: 200


In [3]:
# Day 13: Packet Scanners
@dataclass
class Scanner:
    range: int
    scanner: int = 0
    direction: int = 1

    def tick(self):
        self.scanner += self.direction
        if self.scanner == 0 or self.scanner == self.range - 1:
            self.direction = -self.direction


def hit_severity(scanners, pos):
    if scanners.get(pos) and scanners[pos].scanner == 0:
        # print(f"Hit: {pos}*{scanners[pos].range}")
        return pos * scanners[pos].range
    return 0


def tick_all(scanners):
    for scanner in scanners.values():
        scanner.tick()


def trip(scanners):
    severity = pos = 0
    for pos in range(max(scanners) + 1):
        severity += hit_severity(scanners, pos)
        tick_all(scanners)
    return severity


def safe_delay(cycle_times):
    # Cycle length of a scanner = (range-1)*2
    # You get hit if scanner is in position 0, i.e. (delay + offset) % cycle == 0
    for delay in range(10**9):
        if all((delay + offset) % cycle != 0 for offset, cycle in cycle_times.items()):
            return delay


lines = open("2017/13.txt").read().strip().splitlines()
scanners = {}
cycle_times = {}
for line in lines:
    depth, _range = parse("{:d}: {:d}", line).fixed
    scanners[depth] = Scanner(_range)
    cycle_times[depth] = (_range - 1) * 2

print("Part 1:", trip(scanners))  # 2384
print("Part 2:", safe_delay(cycle_times))  # 3921270

Part 1: 2384
Part 2: 3921270


In [15]:
# Day 14: Disk Defragmentation
def grid(hash):
    result = []
    for row in range(128):
        hex_number = knot_hash(hash + "-" + str(row))  # Knot hash implemented on day 10
        bits = f"{int(hex_number, 16):0128b}"
        for column, used in enumerate(bits):
            if used == "1":
                result.append((row, column))
    return result


def neighbors(point, grid):
    r, c = point
    candidates = [(r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)]
    return [node for node in candidates if node in grid]


def graph(grid):
    result = nx.Graph()
    for node in grid:
        result.add_node(node)
        for neighbor in neighbors(node, grid):
            result.add_edge(node, neighbor)
    return result


# Code above can be replaced with graph2 function. Cuts execution time from 7s to 2.5s
def graph2(hash):
    "Create graph directly from hash without grid intermediate. See https://www.reddit.com/r/adventofcode/comments/7jpelc/comment/dr8dpjj/"
    graph = nx.grid_2d_graph(128, 128)
    for row in range(128):
        hex_number = knot_hash(hash + "-" + str(row))  # Knot hash implemented on day 10
        bits = f"{int(hex_number, 16):0128b}"
        for column, used in enumerate(bits):
            if used == "0":
                graph.remove_node((row, column))
    return graph


hash = "amgozmfv"
# graph = graph(grid(hash))
graph = graph2(hash)
print("Part 1:", nx.number_of_nodes(graph))  # 8222
print("Part 2:", nx.number_connected_components(graph))  # 1086

Part 1: 8222
Part 2: 1086


In [7]:
# Day 15: Dueling Generators
@jit
def values(number, factor, mask=0, divisor=2147483647):
    "Yields a value if the bits in `mask` are zero. If using e.g. 3 as a mask, this will yield values that are evenly divisible by 4."
    while True:
        number = (number * factor) % divisor
        if number & mask == 0:
            yield number


a_factor = 16807
b_factor = 48271
a_seed = 699
b_seed = 124
a_mask = 3
b_mask = 7

part1 = sum(
    1
    for a, b, _ in zip(
        values(a_seed, a_factor), values(b_seed, b_factor), range(40_000_000)
    )
    if a & 0xFFFF == b & 0xFFFF
)

part2 = sum(
    1
    for a, b, _ in zip(
        values(a_seed, a_factor, a_mask),
        values(b_seed, b_factor, b_mask),
        range(5_000_000),
    )
    if a & 0xFFFF == b & 0xFFFF
)

# Runs in 45 seconds without jit, 12 seconds with
print("Part 1:", part1)  # 600
print("Part 2:", part2)  # 313

Part 1: 600
Part 2: 313


In [65]:
# Day 16: Permutation Promenade
def swap(programs, a: int, b: int):
    programs[a], programs[b] = programs[b], programs[a]


def swap_programs(programs, a: str, b: str):
    ia = programs.index(a)
    ib = programs.index(b)
    swap(programs, ia, ib)


def execute_steps(programs, steps):
    for step in steps:
        if p := parse("s{:d}", step):
            spin = p[0]
            programs.rotate(spin)
        elif p := parse("x{:d}/{:d}", step):
            a, b = p
            swap(programs, a, b)
        elif p := parse("p{}/{}", step):
            a, b = p
            swap_programs(programs, a, b)
    return programs


steps = tuple(open("2017/16.txt").read().strip().split(","))
programs = deque("abcdefghijklmnop")
print("Part 1:", "".join(execute_steps(programs, steps)))  # ebjpfdgmihonackl

Part 1: ebjpfdgmihonackl


In [51]:
# After 30 steps we are back to where we started
for i in range(2, 31):
    print(f"Step {i:02}:", "".join(execute_steps(programs, steps)))

Step 02: dblfiegmjknpohac
Step 03: anbfdigmcokeplhj
Step 04: eocfhpgainmdkljb
Step 05: fbpiengdjmoaklch
Step 06: dnjceogmlphfkbia
Step 07: pkjfebgacinhmold
Step 08: aojifkgehcnbdlpm
Step 09: apljchgdifnokbme
Step 10: abocefghijklmndp
Step 11: ebjpfcgmihdnaokl
Step 12: cblfiegmjknpdhao
Step 13: anbfcigmodkeplhj
Step 14: edofhpgainmckljb
Step 15: fbpiengcjmdakloh
Step 16: cnjoedgmlphfkbia
Step 17: pkjfebgaoinhmdlc
Step 18: adjifkgehonbclpm
Step 19: apljohgcifndkbme
Step 20: abdoefghijklmncp
Step 21: ebjpfogmihcnadkl
Step 22: oblfiegmjknpchad
Step 23: anbfoigmdckeplhj
Step 24: ecdfhpgainmokljb
Step 25: fbpiengojmcakldh
Step 26: onjdecgmlphfkbia
Step 27: pkjfebgadinhmclo
Step 28: acjifkgehdnbolpm
Step 29: apljdhgoifnckbme
Step 30: abcdefghijklmnop


In [57]:
# After 1,000,000,000 steps we have the same result as in step:
print(1_000_000_000 % 30)
# which is:
print("Part 2:", "abocefghijklmndp")

10
Part 2: abocefghijklmndp


In [152]:
# Day 17: Spinlock
def spinlock(steps=354):
    def insert_value(buffer, pos, value, steps):
        pos = (pos + steps) % len(buffer)
        buffer.insert(pos + 1, value)
        return pos + 1

    buffer = [0]
    pos = 0
    for i in range(1, 2018):
        pos = insert_value(buffer, pos, i, steps)

    return buffer, pos


# Part 2, only care about value after 0. Buffer length == value to insert
def spinlock2(steps=354):
    pos = 0
    result = 0
    for i in range(1, 50_000_000):
        pos = (pos + steps) % i + 1
        if pos == 1:
            result = i
    return result


buffer, pos = spinlock()
print("Part 1:", buffer[pos + 1])  # 2000
print("Part 2:", spinlock2()) # 10242889

Part 1: 2000
Part 2: 10242889


In [5]:
# Day 18: Duet
def run_assembly(pid, pc, registers, instructions, send=None, receive=None):
    def value(x):
        try:
            return int(x)
        except ValueError:
            return registers[x]

    global sent
    freq = 0
    while pc < len(instructions):
        if p := parse("snd {}", instructions[pc]):
            x = p[0]
            if pid == 2:
                freq = value(x)
            else:
                send.append(value(x))
                if pid == 1:
                    sent += 1
        elif p := parse("set {} {}", instructions[pc]):
            x, y = p
            registers[x] = value(y)
        elif p := parse("add {} {}", instructions[pc]):
            x, y = p
            registers[x] += value(y)
        elif p := parse("mul {} {}", instructions[pc]):
            x, y = p
            registers[x] *= value(y)
        elif p := parse("mod {} {}", instructions[pc]):
            x, y = p
            registers[x] %= value(y)
        elif p := parse("rcv {}", instructions[pc]):
            if pid == 2:
                return freq
            try:
                reg = p[0]
                registers[reg] = value(receive.popleft())
            except IndexError:
                # Waiting for value in my queue
                return pc
        elif p := parse("jgz {} {}", instructions[pc]):
            x, y = p
            if value(x) > 0:
                pc += value(y) - 1  # -1 to offset increment for all instructions
        else:
            raise ValueError("Can't parse", instructions[pc])
        pc += 1
    print(f"{pid} - ran off the edge")


def part1():
    registers = defaultdict(int)
    return run_assembly(2, 0, registers, instructions)


def part2():
    def run_both():
        nonlocal pc0, pc1
        pc0 = run_assembly(
            pid=0,
            pc=pc0,
            registers=registers0,
            instructions=instructions,
            send=queue_a,
            receive=queue_b,
        )

        pc1 = run_assembly(
            pid=1,
            pc=pc1,
            registers=registers1,
            instructions=instructions,
            send=queue_b,
            receive=queue_a,
        )

    pc0 = pc1 = 0
    registers0 = defaultdict(int)
    registers1 = defaultdict(int)
    registers0["p"] = 0
    registers1["p"] = 1

    queue_a = deque()
    queue_b = deque()

    run_both()  # Will make sure something is in queue_a and/or queue_b
    while queue_a or queue_b:
        run_both()
    return sent


sent = 0
registers = defaultdict(int)
instructions = open("2017/18.txt").read().strip().splitlines()
print("Part 1:", part1())  # 8600
print("Part 2:", part2())  # 7239

Part 1: 8600
Part 2: 7239


In [39]:
# Day 19: A Series of Tubes
@dataclass
class Turtle:
    row = 0
    column = 1
    dirs = deque(((1, 0), (0, 1), (-1, 0), (0, -1)))  # S, E, N, W

    def forward(self):
        dr, dc = self.dirs[0]
        self.row += dr
        self.column += dc

    def next_row(self):
        dr, _ = self.dirs[0]
        return self.row + dr

    def next_column(self):
        _, dc = self.dirs[0]
        return self.column + dc

    def turn_left(self):
        self.dirs.rotate(-1)

    def turn_right(self):
        self.dirs.rotate(1)


def current(grid, turtle):
    return grid[turtle.row][turtle.column]


def can_continue(grid, turtle):
    return grid[turtle.next_row()][turtle.next_column()] != " "


def next_direction(grid, turtle):
    assert not can_continue(grid, turtle)
    turtle.turn_left()
    # print(f"Turning left, next dir: {turtle.dirs[0]}")
    if not can_continue(grid, turtle):
        turtle.turn_right()
        turtle.turn_right()
        # print(f"Turning right twice, next dir: {turtle.dirs[0]}")


def fast_forward(grid, turtle):
    "Move forward as far as possible, return letters found on the way."
    found_letters = []
    steps = 0
    while can_continue(grid, turtle):
        turtle.forward()
        if re.match(r"\w", current(grid, turtle)):
            found_letters.append(current(grid, turtle))
        steps += 1
    return found_letters, steps


grid = open("2017/19.txt").read().splitlines()
found_letters = []
total_steps = 1  # We count the initial step
turtle = Turtle()

while can_continue(grid, turtle):
    letters, steps = fast_forward(grid, turtle)
    found_letters.extend(letters)
    total_steps += steps
    next_direction(grid, turtle)
print("Part 1:", "".join(found_letters))  # RYLONKEWB
print("Part 2:", total_steps) # 16016

Part 1: RYLONKEWB
Part 2: 16016
